# Lecture 2: Introduction to Language Modeling

In this lecture, we will introduce the concept of language modeling and implement a bigram language model.

Let's make a bigram language model that generates names character by character.

## Bigram Language Model

Bigram language model is a statistical language model that predicts the next token based on the current token.

For example, given the sentence "A cat sat on the mat"
- A -> cat
- cat -> sat
- sat -> on
- on -> the
- the -> mat

For practical reasons, let's use a character-level language model.

### Importing Libraries

In [ ]:
import os
import matplotlib.pyplot as plt
from dataclasses import dataclass
import torch
from torch.nn import functional as F
from utils import load_text, set_seed
%matplotlib inline

### Configuration

In [ ]:
@dataclass
class BigramConfig:
    root_dir: str = os.getcwd() + "/../../"
    dataset_path: str = "data/names.txt"

    # Tokenizer
    vocab_size: int = 0  # Set later

    seed: int = 101

config = BigramConfig()

### Reproducibility

In [ ]:
set_seed(config.seed)
generator = torch.Generator().manual_seed(config.seed)

### Dataset

In [ ]:
# Load text and split by lines
names = load_text(config.root_dir + config.dataset_path).splitlines()

#### Statistics

In [ ]:
################################################################################
# TODO:                                                                        #
# Print some info about the dataset                                            #
# number of names, the first 5 names, min/max name length, etc.                #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

In [ ]:
# Plot name length
plt.hist([len(name) for name in names], bins=20)
plt.xlabel("Name Length")
plt.ylabel("Frequency")
plt.title("Name Length Histogram")
plt.show()

# Plot alphabet frequency
alphabet = [chr(i) for i in range(97, 123)]  # all alphabet characters
alphabet_counts = torch.zeros(len(alphabet))
for name in names:
    for char in name:
        alphabet_counts[ord(char) - 97] += 1
plt.bar(alphabet, alphabet_counts)
plt.xlabel("Alphabet")
plt.ylabel("Frequency")
plt.title("Alphabet Histogram")
plt.show()

In [ ]:
# Plot what alphabet comes after other alphabets
alphabet2idx = {char: idx for idx, char in enumerate(alphabet)}
idx2alphabet = {idx: char for char, idx in alphabet2idx.items()}
alphabet_matrix = torch.zeros(len(alphabet), len(alphabet), dtype=torch.int32)
for name in names:
    for char1, char2 in zip(name, name[1:]):
        alphabet_matrix[alphabet2idx[char1], alphabet2idx[char2]] += 1
plt.figure(figsize=(16, 16))
plt.imshow(alphabet_matrix, cmap='Blues')
for i in range(len(alphabet)):
    for j in range(len(alphabet)):
        chstr = idx2alphabet[i] + idx2alphabet[j]
        plt.text(j, i, chstr, ha='center', va='bottom', color='gray')
        plt.text(j, i, alphabet_matrix[i, j].item(), ha='center', va='top', color='gray')
plt.axis('off')
plt.show()
# This is the statistics we will use to build the bigram language model.

### Preprocessing

#### 1. Add special tokens

The matrix above has a problem. It does not indicate the beginning and end of a name.
- The model does not know what alphabet to start with.
- The model does not know when to stop generating a name.

To indicate the beginning and end of a name, we need to add special tokens.
- Beginning: `<S>`
- End: `<E>`

For example, the name "emma" will be "`<S>`emma`<E>`"

In [ ]:
# Add special token
names = ["<S>" + name + "<E>" for name in names]
print(f"First 5 names: {names[:5]}")

Let's use the **same** special token for the beginning and end of a name.

Why? Those two tokens will not get mixed up.

In [ ]:
# Reload the data
names = load_text(config.root_dir + config.dataset_path).splitlines()

In [ ]:
################################################################################
# TODO:                                                                        #
# Add the special token '.' to the beginning and end of each name.             #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

#### 2. Tokenizer

To feed the names to the model, we need to convert them to integers. Tokenization is the process of converting text to integers.

We will use a character-level tokenizer. ('.' -> 0, 'a' -> 1, 'b' -> 2 , ..., 'z' -> 26)

In [ ]:
chars = [chr(i) for i in range(97, 123)]  # all alphabet characters
chars.insert(0, ".")  # Add special token
config.vocab_size = len(chars)
print(f"Characters: {chars}")
print(f"Vocab size: {config.vocab_size}")

In [ ]:
# Create mapping
str2idx = {char: idx for idx, char in enumerate(chars)}
idx2str = {idx: char for char, idx in str2idx.items()}
print(f"str2idx: {str2idx}")
print(f"idx2str: {idx2str}")

### Part 1: Statistical approach

#### Statistics

What alphabet is likely to come after another alphabet?

In [ ]:
# Create a tensor to store what alphabet comes after other alphabets
char_matrix = torch.zeros(config.vocab_size, config.vocab_size, dtype=torch.int32)
for name in names:
    for char1, char2 in zip(name, name[1:]):
        char_matrix[str2idx[char1], str2idx[char2]] += 1
plt.figure(figsize=(16, 16))
plt.imshow(char_matrix, cmap='Blues')
for i in range(config.vocab_size):
    for j in range(config.vocab_size):
        stat = idx2str[i] + idx2str[j]
        plt.text(j, i, stat, ha='center', va='bottom', color='gray')
        plt.text(j, i, char_matrix[i, j].item(), ha='center', va='top', color='gray')
plt.axis('off')
plt.show()

In [ ]:
################################################################################
# TODO:                                                                        #
# Get the probabilities by dividing the counts by the total counts             #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

In [ ]:
# Let's plot the probabilities for the first alphabet
plt.bar([idx2str[i] for i in range(config.vocab_size)], prob[:, 0])
plt.xlabel("Alphabet")
plt.ylabel("Probability")
plt.title("First Alphabet Probability")
plt.show()
print(prob[:, 0])

#### Inference

In [ ]:
def generate_name(probs):
    new_name = []
    start_idx = str2idx["."]
    
    while True:
        # Sample
        print("------------------")
        print(f"Input: {idx2str[start_idx]}")
        print(f"Probabilities: {probs[start_idx]}")
        next_idx = torch.multinomial(probs[start_idx], num_samples=1, generator=generator).item()
        
        # Decode
        new_char = idx2str[next_idx]
        print(f"Output (probability): {new_char} ({probs[start_idx, next_idx]:.4f})")
        new_name.append(new_char)
        
        # Update
        start_idx = next_idx
        
        if next_idx == str2idx["."]:
            break
            
    return ''.join(new_name)

In [ ]:
# Generate a random name
uniform_prob = torch.ones(config.vocab_size, config.vocab_size) / config.vocab_size
for _ in range(1):
    print(generate_name(uniform_prob))

In [ ]:
# Generate a name based on the probabilities
for _ in range(1):
    print(generate_name(prob))

#### Evaluation

How should we evaluate the quality of the generated names?

In [ ]:
################################################################################
# TODO:                                                                        #
# What is the probability of generating a random new token?                    #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
print(f"1 / vocab size: {random_prob}")

In [ ]:
for name in names[:3]:
    for char1, char2 in zip(name, name[1:]):
        char1_id = str2idx[char1]
        char2_id = str2idx[char2]
        print(f"'{char1}' -> '{char2}': P={prob[char1_id, char2_id]:.4f}")

Intuitively, we want to maximize the probability of generating the correct next token given the current token.
- Not learned: 1 / vocab_size = 0.037
- If learned, higher the better (should be higher than 0.037)

How can we summarize into a single number, that shows the quality of the model?

**Maximum Likelihood Estimation (MLE)**: Product of all probabilities
- Why use logarithm? multiplication -> addition



In [ ]:
for name in names[:3]:
    for char1, char2 in zip(name, name[1:]):
        char1_id = str2idx[char1]
        char2_id = str2idx[char2]
        print(f"'{char1}' -> '{char2}': P={prob[char1_id, char2_id]:.4f}. LogP={torch.log(prob[char1_id, char2_id]):.4f}")

In [ ]:
# Log likelihood of first 3 names
# log(a*b) = log(a) + log(b)
log_likelihood = 0
n = 0
for name in names[:3]:
    for char1, char2 in zip(name, name[1:]):
        char1_id = str2idx[char1]
        char2_id = str2idx[char2]
        n += 1
        ################################################################################
        # TODO:                                                                        #
        # Log likelihood of the correct next token given the current token             #
        ################################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
print(f"Average Log likelihood: {-log_likelihood / n:.4f}")
# We designed the loss function. yay!

**Summary of what we've done so far**

Bigram Language Model
- Parameters: Tensor of size (vocab_size, vocab_size) -> Statistical approach
- Evaluation: Average Log likelihood of the correct next token given the current token


### Part 2: Neural Network approach

This time, we will use gradient descent and backpropagation to learn the probabilities.

#### Dataloader

We need to create a dataset of (Input, Target) pairs.
- Input: Current token
- Target: Next token

In [ ]:
# Example of Input, Target pairs
example_name = names[0] # emma.
print(f"Name: {example_name}")

example_input0 = str2idx["."]  # 0
example_target0 = str2idx["e"]  # 5
print(f"Input: {example_input0}, Target: {example_target0}")

example_input1 = str2idx["e"]  # 5
example_target1 = str2idx["m"]  # 13
print(f"Input: {example_input1}, Target: {example_target1}")

In [ ]:
# Set of Input, Target pairs
inputs, targets = [], []
for name in names:
    for char1, char2 in zip(name, name[1:]):
        ################################################################################
        # TODO:                                                                        #
        # Append the input string and target string to the inputs and targets list.    #
        ################################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
# Convert to tensor
inputs = torch.tensor(inputs, dtype=torch.long)
targets = torch.tensor(targets, dtype=torch.long)

print(f"Input shape: {inputs.shape}")
print(f"Target shape: {targets.shape}")
print(f"First (Input, Target): ({inputs[0]}, {targets[0]})")
print(f"Second (Input, Target): ({inputs[1]}, {targets[1]})")

#### One-hot Encoding

The model cannot take integers as input. (e.g., Input: 5, Target: 13)

The model expects a vector. One way to convert integers to vectors is to use one-hot encoding.
- Input: 5 -> [0, 0, 0, 0, 0, 1, 0, ..., 0]
- Target: 1 -> [0, 1, 0, 0, ..., 0]

PyTorch provides One-Hot Encoding functionality. [PyTorch Documentation](https://pytorch.org/docs/stable/generated/torch.nn.functional.one_hot.html)

Read the documentation and implement the one-hot encoding.

In [ ]:
# One-hot encoding
################################################################################
# TODO:                                                                        #
# Implement one-hot encoding for inputs and targets.                          #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

# Convert data type to float
inputs_encoded = inputs_encoded.float()
targets_encoded = targets_encoded.float()

print(f"Input shape: {inputs_encoded.shape}")
print(f"Target shape: {targets_encoded.shape}")
print(f"Input dtype: {inputs_encoded.dtype}")
print(f"Target dtype: {targets_encoded.dtype}")
print(f"First Input: {inputs_encoded[0]}")
print(f"First Target: {targets_encoded[0]}")

#### Model

Let's define the weights for a simple linear model.

y = Wx + b
- x: (data_size, vocab_size)
- W: (vocab_size, vocab_size)
- b: (vocab_size)
- y: (data_size, vocab_size)

What is torch.randn? [PyTorch Documentation](https://pytorch.org/docs/stable/generated/torch.randn.html)

Why add requires_grad=True?
- If the tensor has requires_grad=False, PyTorch will not track operations on it, and it will not be able to calculate gradients. (saving memory)

In [ ]:
# Initialize weights
W = torch.randn(config.vocab_size, config.vocab_size, generator=generator, requires_grad=True)
b = torch.randn(config.vocab_size, generator=generator, requires_grad=True)

print(f"W shape: {W.shape}")
print(f"b shape: {b.shape}")

#### Training

In [ ]:
# Example of forward pass
# data_size = 1
logits = torch.matmul(inputs_encoded[0], W) + b  # or use @
print(f"Input: {inputs_encoded[0]}")
print(f"Logits shape: {logits.shape}")
print(f"Logits: {logits}")

In [ ]:
# TODO: Training loop

Q: What do we do to get the probability of an output?

A: **Softmax**
- Softmax converts logits to probabilities.
- Softmax: exp(x) / sum(exp(x))

In [ ]:
# Probability matrix
prob = logits.exp() / logits.exp().sum(dim=-1, keepdim=True)
print(f"Probability shape: {prob.shape}")
print(f"Probability: {prob}")
print(f"Sum of Probability: {prob.sum()}")

#### Inference

In [ ]:
# Example prediction
next_idx = torch.multinomial(prob, num_samples=1, generator=generator).item()
print(f"Input: {inputs_encoded[0]}")
print(f"Output (probability): {next_idx} ({prob[next_idx]:.4f})")

In [ ]:
# TODO: Generate a name